<a href="https://colab.research.google.com/github/bbookarisara/Interactive-Dashboard-For-Business-Decision/blob/main/I2BLabel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Labeling

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
path = '/content/Data_withoutBillNo_Mapping.csv'
df = pd.read_csv(path)

<ipython-input-14-5974eae34f61>:6: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [ ]:
import pandas as pd
import re

def assign_group(df):
    # Define conditions for grouping
    def classify_item(item, unit):
        if any(keyword in item for keyword in ['แช่แข็ง', 'สด', 'กุ้ง', 'ดอกไม้']) or 'โฟม' in unit:
            return 'กล่องเย็น'
        elif any(keyword in item for keyword in ['จดหมาย', 'เอกสาร']) or any(keyword in unit for keyword in ['ซอง', 'ฉบับ']):
            return 'เอกสาร'
        elif 'เหมา' in item or 'คัน' in unit or 'เหมา' in unit:
            return 'เหมา'
        elif 'กันกระแทก' in item or 'กันกระแทก' in unit:
            return 'กล่องกันกระแทก'
        else:
            return 'กล่อง'

    # Apply the classification function to each row
    df['PACKAGE'] = df.apply(lambda row: classify_item(row['ITEM_DESCแสดงหน้าบิล'], row['UNIT_NAME']), axis=1)
    return df

In [ ]:
df_labeled = assign_group(df)

TypeError: argument of type 'float' is not iterable

In [ ]:
df_labeled['PACKAGE'].value_counts()

,count
PACKAGE,
กล่อง,724071
กล่องเย็น,97484
เอกสาร,9530
เหมา,4081
กล่องกันกระแทก,67


In [ ]:
df_labeled = df_labeled[['BILL_INDEX','BILL_DATE','ITEM_DESCแสดงหน้าบิล','PACKAGE']]
df_labeled

,BILL_INDEX,BILL_DATE,ITEM_DESCแสดงหน้าบิล,PACKAGE
0,0,20240301,เอกสารโอทีขนส่ง+โอทียานยนต์+เบี้ยเลี้ยงนอกสถาน...,เอกสาร
1,1,20240301,เอกสาร ...,เอกสาร
2,2,20240301,เอกสาร ...,เอกสาร
3,3,20240301,เอกสารใบสมัครงาน ...,เอกสาร
4,4,20240301,สินค้าตกค้าง/กันกระแทก ...,กล่องกันกระแทก
...,...,...,...,...
835228,1184918,20240331,ของสด ...,กล่องเย็น
835229,1184920,20240331,ของสด ...,กล่องเย็น
835230,1184921,20240331,ของสด ...,กล่องเย็น
835231,1184923,20240331,ของสด ...,กล่องเย็น


In [ ]:
df_labeled.to_csv('df_labeled.csv', index=False)

Size Allocation


In [ ]:
#drop irrelevant column
df_2 = df.drop(['ITEM_NAMEในระบบ','CONSIGNEE_ZIPCODE', 'ITEM_CODE','CONSIGNEE_TAMBON','CONSIGNEE_AMPHUR', 'CONSIGNEE_PROVINCE','DEST_CODE','REG_Code'], axis=1)
df_2

,BILL_INDEX,BILL_DATE,ITEM_DESCแสดงหน้าบิล,QTY,UNIT_NAME
0,0,20240301,เอกสารโอทีขนส่ง+โอทียานยนต์+เบี้ยเลี้ยงนอกสถาน...,1,ซอง ...
1,1,20240301,เอกสาร ...,1,ซอง ...
2,2,20240301,เอกสาร ...,1,ซอง ...
3,3,20240301,เอกสารใบสมัครงาน ...,1,ซอง ...
4,4,20240301,สินค้าตกค้าง/กันกระแทก ...,14,ห่อ ...
...,...,...,...,...,...
77845,110628,20240201,ผ้า ...,18.0,ห่อ ...
77846,110629,20240201,ผ้า ...,2.0,ห่อ ...
77847,110630,20240201,ผ้า ...,1.0,ห่อ ...
77848,110631,20240201,ผ้า ...,1.0,ห่อ ...


In [ ]:
import re

def extract_info(text):
  sizes = re.findall(r"(\d+\*\d+\*\d+)", text)
  weights = re.findall(r"(\d+)\s*kg", text)

  return  sizes, weights

#add columns
df_2['size (cm)'] = ''
df_2['weight (kg)'] = ''

In [ ]:
#if there is info -> fill
for index, row in df_2.iterrows():
  sizes, weights = extract_info(row['ITEM_DESCแสดงหน้าบิล'])

  df_2.at[index, 'size (cm)'] = ', '.join(sizes) if sizes else ''
  df_2.at[index, 'weight (kg)'] = ', '.join(weights) if weights else ''

df_2

,BILL_INDEX,BILL_DATE,ITEM_DESCแสดงหน้าบิล,QTY,UNIT_NAME,size (cm),weight (kg)
0,0,20240301,เอกสารโอทีขนส่ง+โอทียานยนต์+เบี้ยเลี้ยงนอกสถาน...,1,ซอง ...,,
1,1,20240301,เอกสาร ...,1,ซอง ...,,
2,2,20240301,เอกสาร ...,1,ซอง ...,,
3,3,20240301,เอกสารใบสมัครงาน ...,1,ซอง ...,,
4,4,20240301,สินค้าตกค้าง/กันกระแทก ...,14,ห่อ ...,,
...,...,...,...,...,...,...,...
77845,110628,20240201,ผ้า ...,18.0,ห่อ ...,,
77846,110629,20240201,ผ้า ...,2.0,ห่อ ...,,
77847,110630,20240201,ผ้า ...,1.0,ห่อ ...,,
77848,110631,20240201,ผ้า ...,1.0,ห่อ ...,,


In [ ]:
df_complete =  df_2[(df_2['size (cm)'] != '') & (df_2['weight (kg)'] != '')]
df_complete

,BILL_INDEX,BILL_DATE,ITEM_DESCแสดงหน้าบิล,QTY,UNIT_NAME,size (cm),weight (kg)
268,414,20240301,ของสด กล่องโฟมใหญ่ 60*46*30 cm / 20 kg ...,1,โฟม ญ ...,60*46*30,20
269,415,20240301,ของสด กล่องโฟมใหญ่ 60*46*30 cm / 20 kg ...,1,ฟญ ...,60*46*30,20
314,517,20240301,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg ...,1,โฟม ก ...,35*48*35,15
318,523,20240301,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg ...,1,โฟม ก ...,35*48*35,15
325,530,20240301,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg ...,1,โฟม ก ...,35*48*35,15
...,...,...,...,...,...,...,...
77668,110322,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg ...,1.0,โฟมกลาง ...,35*48*35,15
77669,110326,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg-ลูกชิ้น...,1.0,โฟมกลาง ...,35*48*35,15
77672,110334,20240201,ของสด กล่องโฟมจิ๋ว 37*25*21 cm / 2 kg ...,2.0,โฟมเล็ก ...,37*25*21,2
77719,110436,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg-ทูน่า ...,1.0,โฟมกลาง ...,35*48*35,15


In [ ]:
df_incomplete = df_2[~df_2.index.isin(df_complete.index)]
df_incomplete.dropna(inplace=True)
df_incomplete.isna().sum()

<ipython-input-24-d0fe043a2828>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_incomplete.dropna(inplace=True)


,0
BILL_INDEX,0
BILL_DATE,0
ITEM_DESCแสดงหน้าบิล,0
QTY,0
UNIT_NAME,0
size (cm),0
weight (kg),0


In [ ]:
#check wether there is not empty string
df_incomplete[df_incomplete['weight (kg)'] != '']

,BILL_INDEX,BILL_DATE,ITEM_DESCแสดงหน้าบิล,QTY,UNIT_NAME,size (cm),weight (kg)
378,626,20240301,กระเทียม 36x20x10cm / 10kg ...,186,ถุง ...,,10
428,727,20240301,เสื้อผ้า 62x125x40 cm / 60 kg (แพร่) ...,2,ห่อ ...,,60
584,990,20240301,ขนม 42X52X64 cm/ 2 kg (อำเภอเมือง) ...,33,กล่อง ...,,2
590,1008,20240301,ขนม 42X52X64 cm/ 2 kg (อำเภอเมือง) ...,6,กล่อง ...,,2
976,1679,20240301,ผ้า(ทุกชนิด) 45x115x30 cm / 55kg (แพร่) เซรามิ...,7,กล่อง ...,,55
...,...,...,...,...,...,...,...
75643,106939,20240201,กระเทียม 36x20x10cm / 10kg มัดจุก ...,82.0,ถุง ...,,10
75644,106940,20240201,กระเทียม 36x20x10cm / 10kg กลีบ ...,344.0,ถุง ...,,10
75802,107274,20240201,ขนม 42X52X64 cm/ 2 kg (อำเภอเมือง) ...,7.0,กล่อง ...,,2
76754,108928,20240201,ขนม 10 kg (ต่างอำเภอ) ...,1.0,กล่อง ...,,10


In [ ]:
df_test = df_incomplete.drop(columns=['size (cm)','weight (kg)'])
df_test

,BILL_INDEX,BILL_DATE,ITEM_DESCแสดงหน้าบิล,QTY,UNIT_NAME
0,0,20240301,เอกสารโอทีขนส่ง+โอทียานยนต์+เบี้ยเลี้ยงนอกสถาน...,1,ซอง ...
1,1,20240301,เอกสาร ...,1,ซอง ...
2,2,20240301,เอกสาร ...,1,ซอง ...
3,3,20240301,เอกสารใบสมัครงาน ...,1,ซอง ...
4,4,20240301,สินค้าตกค้าง/กันกระแทก ...,14,ห่อ ...
...,...,...,...,...,...
77844,110624,20240201,อะไหล่ ...,1.0,กล่อง ...
77845,110628,20240201,ผ้า ...,18.0,ห่อ ...
77846,110629,20240201,ผ้า ...,2.0,ห่อ ...
77847,110630,20240201,ผ้า ...,1.0,ห่อ ...


In [ ]:
#split size column
def split_size_column(df):
  # First, check if 'size (cm)' has values that can be split into 3 parts
  split_columns = df['size (cm)'].str.split('*', expand=True)

  # Now, safely assign the split columns, ensuring all rows have 3 values
  df[['กว้าง (cm)', 'ยาว (cm)', 'สูง (cm)']] = split_columns

  # Convert the new columns to numeric type
  df['กว้าง (cm)'] = pd.to_numeric(df_complete['กว้าง (cm)'], errors='coerce')
  df['ยาว (cm)'] = pd.to_numeric(df_complete['ยาว (cm)'], errors='coerce')
  df['สูง (cm)'] = pd.to_numeric(df_complete['สูง (cm)'], errors='coerce')

  # Drop the original 'size (cm)' column if it's no longer needed
  df = df.drop(columns=['size (cm)'])
  return df

In [ ]:
df_train = split_size_column(df_complete)
df_train

<ipython-input-23-cf3876097192>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['กว้าง (cm)', 'ยาว (cm)', 'สูง (cm)']] = split_columns
<ipython-input-23-cf3876097192>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['กว้าง (cm)'] = pd.to_numeric(df_complete['กว้าง (cm)'], errors='coerce')
<ipython-input-23-cf3876097192>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

,BILL_INDEX,BILL_DATE,ITEM_DESCแสดงหน้าบิล,QTY,UNIT_NAME,weight (kg),กว้าง (cm),ยาว (cm),สูง (cm)
268,414,20240301,ของสด กล่องโฟมใหญ่ 60*46*30 cm / 20 kg ...,1,โฟม ญ ...,20,60,46,30
269,415,20240301,ของสด กล่องโฟมใหญ่ 60*46*30 cm / 20 kg ...,1,ฟญ ...,20,60,46,30
314,517,20240301,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg ...,1,โฟม ก ...,15,35,48,35
318,523,20240301,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg ...,1,โฟม ก ...,15,35,48,35
325,530,20240301,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg ...,1,โฟม ก ...,15,35,48,35
...,...,...,...,...,...,...,...,...,...
77668,110322,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg ...,1.0,โฟมกลาง ...,15,35,48,35
77669,110326,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg-ลูกชิ้น...,1.0,โฟมกลาง ...,15,35,48,35
77672,110334,20240201,ของสด กล่องโฟมจิ๋ว 37*25*21 cm / 2 kg ...,2.0,โฟมเล็ก ...,2,37,25,21
77719,110436,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg-ทูน่า ...,1.0,โฟมกลาง ...,15,35,48,35


In [ ]:
#clean data before training
def clean_training_data(df):
  # Ensure numerical columns are floats
  df['weight (kg)'] = pd.to_numeric(df['weight (kg)'], errors='coerce')
  df['กว้าง (cm)'] = pd.to_numeric(df['กว้าง (cm)'], errors='coerce')
  df['ยาว (cm)'] = pd.to_numeric(df['ยาว (cm)'], errors='coerce')
  df['สูง (cm)'] = pd.to_numeric(df['สูง (cm)'], errors='coerce')

  # Check for any remaining non-numeric values and handle them
  df = df.dropna(subset=['weight (kg)', 'กว้าง (cm)', 'ยาว (cm)', 'สูง (cm)'])
  return df
df_train = clean_training_data(df_train)
df_train

,BILL_INDEX,BILL_DATE,ITEM_DESCแสดงหน้าบิล,QTY,UNIT_NAME,weight (kg),กว้าง (cm),ยาว (cm),สูง (cm)
268,414,20240301,ของสด กล่องโฟมใหญ่ 60*46*30 cm / 20 kg ...,1,โฟม ญ ...,20,60,46,30
269,415,20240301,ของสด กล่องโฟมใหญ่ 60*46*30 cm / 20 kg ...,1,ฟญ ...,20,60,46,30
314,517,20240301,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg ...,1,โฟม ก ...,15,35,48,35
318,523,20240301,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg ...,1,โฟม ก ...,15,35,48,35
325,530,20240301,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg ...,1,โฟม ก ...,15,35,48,35
...,...,...,...,...,...,...,...,...,...
77668,110322,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg ...,1.0,โฟมกลาง ...,15,35,48,35
77669,110326,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg-ลูกชิ้น...,1.0,โฟมกลาง ...,15,35,48,35
77672,110334,20240201,ของสด กล่องโฟมจิ๋ว 37*25*21 cm / 2 kg ...,2.0,โฟมเล็ก ...,2,37,25,21
77719,110436,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg-ทูน่า ...,1.0,โฟมกลาง ...,15,35,48,35


In [ ]:
X = df_train['ITEM_DESCแสดงหน้าบิล']+df_train['UNIT_NAME']
y = df_train[['weight (kg)','กว้าง (cm)','ยาว (cm)','สูง (cm)']]

In [ ]:
X.shape

(1102,)

In [ ]:
!pip install pythainlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 52.7 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from pythainlp.tokenize import word_tokenize
tfidf = TfidfVectorizer(tokenizer=word_tokenize, ngram_range=(1, 2), sublinear_tf=True)
# Fit and transform your text data
X_train_tfidf = tfidf.fit_transform(X_train)  # X_train should have shape (881, some_number_of_features)
X_test_tfidf = tfidf.transform(X_test)

In [ ]:
#1
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor  # Example regressor

# Use a regressor/classifier that supports multi-target
RFmodel = MultiOutputRegressor(RandomForestRegressor())
RFmodel.fit(X_train_tfidf, y_train)
RFmodel_pred = RFmodel.predict(X_test_tfidf)

In [ ]:
#2
from sklearn.linear_model import ElasticNet
# Create and train the multi-target regression model (ElasticNet)
ENmodel = MultiOutputRegressor(ElasticNet(alpha=0.5, l1_ratio=0.5), n_jobs=5)
ENmodel.fit(X_train_tfidf, y_train)
ENmodel_pred = ENmodel.predict(X_test_tfidf)

In [ ]:
#3
from sklearn.tree import DecisionTreeRegressor
# Create and train the decision tree regressor model
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train_tfidf, y_train)
tree_model_pred = tree_model.predict(X_test_tfidf)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Evaluate the model
print(f'RandomForestRegressor MSE: {mean_absolute_error(y_test, RFmodel_pred)}')
print(f'ElasticNet MSE: {mean_absolute_error(y_test, ENmodel_pred)}')
print(f'DecisionTreeRegressor MSE: {mean_absolute_error(y_test, tree_model_pred)}')

RandomForestRegressor MSE: 0.36421945701357467
ElasticNet MSE: 9.975545284714801
DecisionTreeRegressor MSE: 0.22850678733031676


In [ ]:
#test
test_features = df_test['ITEM_DESCแสดงหน้าบิล']+df_test['UNIT_NAME']
test_features_tfidf = tfidf.transform(test_features)
prediction = tree_model.predict(test_features_tfidf)

In [ ]:
df_predictions = pd.DataFrame(prediction, columns=['weight (kg)', 'กว้าง (cm)', 'ยาว (cm)', 'สูง (cm)'])
df_predictions

,weight (kg),กว้าง (cm),ยาว (cm),สูง (cm)
0,17.0,51.0,61.0,52.0
1,17.0,51.0,61.0,52.0
2,17.0,51.0,61.0,52.0
3,17.0,51.0,61.0,52.0
4,17.0,51.0,61.0,52.0
...,...,...,...,...
76742,17.0,50.0,72.0,42.0
76743,17.0,51.0,61.0,52.0
76744,17.0,51.0,61.0,52.0
76745,17.0,51.0,61.0,52.0


In [ ]:
# Split the 'size (cm)' column into three separate columns: กว้าง (cm), ยาว (cm), สูง (cm)
split_columns = df_incomplete['size (cm)'].str.split('*', expand=True)

# Assign the split columns to the respective new columns
df_incomplete['กว้าง (cm)'] = split_columns[0]
df_incomplete['ยาว (cm)'] = split_columns[1]
df_incomplete['สูง (cm)'] = split_columns[2]

df_incomplete = df_incomplete.drop('size (cm)',axis=1)
# View the modified dataframe
df_incomplete

<ipython-input-123-8f36b49b0eb8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_incomplete['กว้าง (cm)'] = split_columns[0]
<ipython-input-123-8f36b49b0eb8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_incomplete['ยาว (cm)'] = split_columns[1]
<ipython-input-123-8f36b49b0eb8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

,BILL_INDEX,BILL_DATE,ITEM_DESCแสดงหน้าบิล,QTY,UNIT_NAME,weight (kg),กว้าง (cm),ยาว (cm),สูง (cm)
0,0,20240301,เอกสารโอทีขนส่ง+โอทียานยนต์+เบี้ยเลี้ยงนอกสถาน...,1,ซอง ...,,,None,None
1,1,20240301,เอกสาร ...,1,ซอง ...,,,None,None
2,2,20240301,เอกสาร ...,1,ซอง ...,,,None,None
3,3,20240301,เอกสารใบสมัครงาน ...,1,ซอง ...,,,None,None
4,4,20240301,สินค้าตกค้าง/กันกระแทก ...,14,ห่อ ...,,,None,None
...,...,...,...,...,...,...,...,...,...
77844,110624,20240201,อะไหล่ ...,1.0,กล่อง ...,,,None,None
77845,110628,20240201,ผ้า ...,18.0,ห่อ ...,,,None,None
77846,110629,20240201,ผ้า ...,2.0,ห่อ ...,,,None,None
77847,110630,20240201,ผ้า ...,1.0,ห่อ ...,,,None,None


In [ ]:
# Assuming df_incomplete and df_predictions are the dataframes

# Columns to update
columns_to_update = ['weight (kg)', 'กว้าง (cm)', 'ยาว (cm)', 'สูง (cm)']

# Iterate over the columns to update
for column in columns_to_update:
    # Replace missing values (None or empty) in df_incomplete with values from df_predictions
    df_incomplete[column] = df_incomplete[column].combine_first(df_predictions[column])

# View the updated df_incomplete
df_incomplete


,BILL_INDEX,BILL_DATE,ITEM_DESCแสดงหน้าบิล,QTY,UNIT_NAME,weight (kg),กว้าง (cm),ยาว (cm),สูง (cm)
0,0,20240301,เอกสารโอทีขนส่ง+โอทียานยนต์+เบี้ยเลี้ยงนอกสถาน...,1,ซอง ...,,,61.0,52.0
1,1,20240301,เอกสาร ...,1,ซอง ...,,,61.0,52.0
2,2,20240301,เอกสาร ...,1,ซอง ...,,,61.0,52.0
3,3,20240301,เอกสารใบสมัครงาน ...,1,ซอง ...,,,61.0,52.0
4,4,20240301,สินค้าตกค้าง/กันกระแทก ...,14,ห่อ ...,,,61.0,52.0
...,...,...,...,...,...,...,...,...,...
77844,110624,20240201,อะไหล่ ...,1.0,กล่อง ...,,,None,None
77845,110628,20240201,ผ้า ...,18.0,ห่อ ...,,,None,None
77846,110629,20240201,ผ้า ...,2.0,ห่อ ...,,,None,None
77847,110630,20240201,ผ้า ...,1.0,ห่อ ...,,,None,None


In [ ]:
df_incomplete.dropna(inplace=True)
df_incomplete.isna().sum()

,0
BILL_INDEX,0
BILL_DATE,0
ITEM_DESCแสดงหน้าบิล,0
QTY,0
UNIT_NAME,0
weight (kg),0
กว้าง (cm),0
ยาว (cm),0
สูง (cm),0


In [ ]:
df_combined = pd.concat([df_complete, df_incomplete])
df_combined = df_combined.sort_index()
df_combined = df_combined.drop('size (cm)',axis=1)
df_combined

,BILL_INDEX,BILL_DATE,ITEM_DESCแสดงหน้าบิล,QTY,UNIT_NAME,weight (kg),กว้าง (cm),ยาว (cm),สูง (cm)
0,0,20240301,เอกสารโอทีขนส่ง+โอทียานยนต์+เบี้ยเลี้ยงนอกสถาน...,1,ซอง ...,,,61.0,52.0
1,1,20240301,เอกสาร ...,1,ซอง ...,,,61.0,52.0
2,2,20240301,เอกสาร ...,1,ซอง ...,,,61.0,52.0
3,3,20240301,เอกสารใบสมัครงาน ...,1,ซอง ...,,,61.0,52.0
4,4,20240301,สินค้าตกค้าง/กันกระแทก ...,14,ห่อ ...,,,61.0,52.0
...,...,...,...,...,...,...,...,...,...
77668,110322,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg ...,1.0,โฟมกลาง ...,15,35,48,35
77669,110326,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg-ลูกชิ้น...,1.0,โฟมกลาง ...,15,35,48,35
77672,110334,20240201,ของสด กล่องโฟมจิ๋ว 37*25*21 cm / 2 kg ...,2.0,โฟมเล็ก ...,2,37,25,21
77719,110436,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg-ทูน่า ...,1.0,โฟมกลาง ...,15,35,48,35


In [ ]:
df_combined.isna().sum()

,0
BILL_DATE,0
ITEM_DESCแสดงหน้าบิล,0
QTY,0
UNIT_NAME,0
PACKAGE,0
weight (kg),0
กว้าง (cm),0
ยาว (cm),0
สูง (cm),0


In [ ]:
path = '/content/df_labeled.csv'
df_labeled = pd.read_csv(path)

In [ ]:
df_labeled = df_labeled[["BILL_INDEX","PACKAGE"]]
df_labeled

,BILL_INDEX,PACKAGE
0,0,เอกสาร
1,1,เอกสาร
2,2,เอกสาร
3,3,เอกสาร
4,4,กล่องกันกระแทก
...,...,...
278407,395077,กล่องเย็น
278408,395078,กล่องเย็น
278409,395079,กล่อง
278410,395080,กล่อง


In [ ]:
import pandas as pd

# Example dataframes
# df_combined = pd.DataFrame({...})
# df_labeled = pd.DataFrame({...})

# Perform the left join
df_merged = df_combined.merge(df_labeled, on='BILL_INDEX', how='left')

df_merged


,BILL_INDEX,BILL_DATE,ITEM_DESCแสดงหน้าบิล,QTY,UNIT_NAME,weight (kg),กว้าง (cm),ยาว (cm),สูง (cm),PACKAGE
0,0,20240301,เอกสารโอทีขนส่ง+โอทียานยนต์+เบี้ยเลี้ยงนอกสถาน...,1,ซอง ...,,,61.0,52.0,เอกสาร
1,1,20240301,เอกสาร ...,1,ซอง ...,,,61.0,52.0,เอกสาร
2,2,20240301,เอกสาร ...,1,ซอง ...,,,61.0,52.0,เอกสาร
3,3,20240301,เอกสารใบสมัครงาน ...,1,ซอง ...,,,61.0,52.0,เอกสาร
4,4,20240301,สินค้าตกค้าง/กันกระแทก ...,14,ห่อ ...,,,61.0,52.0,กล่องกันกระแทก
...,...,...,...,...,...,...,...,...,...,...
76749,110322,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg ...,1.0,โฟมกลาง ...,15,35,48,35,กล่องเย็น
76750,110326,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg-ลูกชิ้น...,1.0,โฟมกลาง ...,15,35,48,35,กล่องเย็น
76751,110334,20240201,ของสด กล่องโฟมจิ๋ว 37*25*21 cm / 2 kg ...,2.0,โฟมเล็ก ...,2,37,25,21,กล่องเย็น
76752,110436,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg-ทูน่า ...,1.0,โฟมกลาง ...,15,35,48,35,กล่องเย็น


In [ ]:
import pandas as pd

# Corrected DataFrame
size_df = pd.DataFrame({
    'type': ['1','2','3','4','5','6','7','8'],
    'weight (kgs)': [0.5, 1.5, 2.5, 5, 10, 15,20, 25],
    'กว้าง (cm)': [35, 33.7, 33.7, 33.7, 33.7, 41.7, 48.1, 54.1],
    'ยาว (cm)': [27.5, 18.3, 32.2, 32.2, 32.2, 35.9, 40.4, 44.4],
    'สูง (cm)': [1, 10, 10, 18, 34.5, 36.9, 38.9, 40.9]
})

size_df

,type,weight (kgs),กว้าง (cm),ยาว (cm),สูง (cm)
0,1,0.5,35.0,27.5,1.0
1,2,1.5,33.7,18.3,10.0
2,3,2.5,33.7,32.2,10.0
3,4,5.0,33.7,32.2,18.0
4,5,10.0,33.7,32.2,34.5
5,6,15.0,41.7,35.9,36.9
6,7,20.0,48.1,40.4,38.9
7,8,25.0,54.1,44.4,40.9


In [ ]:
import pandas as pd

def assign_type(row, size_df):
    # Clean 'QTY' by removing commas and convert to integer
    try:
        qty = int(row['QTY'].replace(',', ''))
    except (ValueError, AttributeError):
        qty = 0  # Handle non-integer or invalid values gracefully

    # Check if PACKAGE is 'เอกสาร'
    if row['PACKAGE'] == 'เอกสาร':
        return '1'

    # Iterate through size_df to find the matching type
    for _, size_row in size_df.iterrows():
        if (row['weight (kg)']  <= size_row['weight (kgs)'] and
            row['กว้าง (cm)']  <= size_row['กว้าง (cm)'] and
            row['ยาว (cm)']  <= size_row['ยาว (cm)'] and
            row['สูง (cm)']  <= size_row['สูง (cm)']):
            return size_row['type']

    # Return 'overweight' if no type fits the record
    return 'overweight'

df_merged['weight (kg)'] = pd.to_numeric(df_merged['weight (kg)'], errors='coerce')
df_merged['กว้าง (cm)'] = pd.to_numeric(df_merged['กว้าง (cm)'], errors='coerce')
df_merged['ยาว (cm)'] = pd.to_numeric(df_merged['ยาว (cm)'], errors='coerce')
df_merged['สูง (cm)'] = pd.to_numeric(df_merged['สูง (cm)'], errors='coerce')


# Apply the function to each row in df_merged
df_merged['type'] = df_merged.apply(assign_type, size_df=size_df, axis=1)

# Display the updated dataframe
df_merged

,BILL_INDEX,BILL_DATE,ITEM_DESCแสดงหน้าบิล,QTY,UNIT_NAME,weight (kg),กว้าง (cm),ยาว (cm),สูง (cm),PACKAGE,type
0,0,20240301,เอกสารโอทีขนส่ง+โอทียานยนต์+เบี้ยเลี้ยงนอกสถาน...,1,ซอง ...,NaN,NaN,61.0,52.0,เอกสาร,1
1,1,20240301,เอกสาร ...,1,ซอง ...,NaN,NaN,61.0,52.0,เอกสาร,1
2,2,20240301,เอกสาร ...,1,ซอง ...,NaN,NaN,61.0,52.0,เอกสาร,1
3,3,20240301,เอกสารใบสมัครงาน ...,1,ซอง ...,NaN,NaN,61.0,52.0,เอกสาร,1
4,4,20240301,สินค้าตกค้าง/กันกระแทก ...,14,ห่อ ...,NaN,NaN,61.0,52.0,กล่องกันกระแทก,overweight
...,...,...,...,...,...,...,...,...,...,...,...
76749,110322,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg ...,1.0,โฟมกลาง ...,15.0,35.0,48.0,35.0,กล่องเย็น,overweight
76750,110326,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg-ลูกชิ้น...,1.0,โฟมกลาง ...,15.0,35.0,48.0,35.0,กล่องเย็น,overweight
76751,110334,20240201,ของสด กล่องโฟมจิ๋ว 37*25*21 cm / 2 kg ...,2.0,โฟมเล็ก ...,2.0,37.0,25.0,21.0,กล่องเย็น,6
76752,110436,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg-ทูน่า ...,1.0,โฟมกลาง ...,15.0,35.0,48.0,35.0,กล่องเย็น,overweight


In [ ]:
import pandas as pd
import numpy as np

def duplicate_records(df):
    # Create an empty list to store repeated rows
    repeated_rows = []

    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        # Convert 'QTY' to string and remove commas
        qty_str = str(row['QTY'])
        qty_str = qty_str.replace(',', '')

        # Convert cleaned string to integer
        try:
            qty = int(qty_str)
        except ValueError:
            qty = 0  # Handle non-integer or invalid values gracefully

        # Append the row repeated `qty` times
        repeated_rows.extend([row] * qty)

    # Create a new DataFrame from the repeated rows
    df_repeated = pd.DataFrame(repeated_rows).reset_index(drop=True)

    return df_repeated

In [ ]:
df_repeat = duplicate_records(df_merged)
df_repeat.value_counts()

,,,,,,,,,,,count
BILL_INDEX,BILL_DATE,ITEM_DESCแสดงหน้าบิล,QTY,UNIT_NAME,weight (kg),กว้าง (cm),ยาว (cm),สูง (cm),PACKAGE,type,
72189,20240311,ไม้เสียบอนามัย 35*60*23 cm / 20 kg (แพร่),110,ห่อเขียว,20.0,35.0,60.0,23.0,กล่อง,overweight,110
89438,20240313,ดอกไม้แช่เย็น 44*90*60cm / 35kg,30,กล่องใหญ่,35.0,44.0,90.0,60.0,กล่องเย็น,overweight,30
537,20240301,ของสด กล่องโฟมใหญ่ 60*46*30 cm / 20 kg,28,ฟญ,20.0,60.0,46.0,30.0,กล่องเย็น,overweight,28
40424,20240307,ของสด กล่องโฟมใหญ่ 60*46*30 cm / 20 kg,25,กล่อง,20.0,60.0,46.0,30.0,กล่องเย็น,overweight,25
56242,20240308,ของสด กล่องโฟมใหญ่ 60*46*30 cm / 20 kg,23,โฟม ญ,20.0,60.0,46.0,30.0,กล่องเย็น,overweight,23
...,...,...,...,...,...,...,...,...,...,...,...
39414,20240306,ดอกไม้แช่เย็น 30*100*35cm / 25kg,1,กล่องเล็ก,25.0,30.0,100.0,35.0,กล่องเย็น,overweight,1
39415,20240306,ดอกไม้แช่เย็น 44*90*60cm / 35kg,1,กล่องใหญ่,35.0,44.0,90.0,60.0,กล่องเย็น,overweight,1
39453,20240306,ของสด กล่องโฟมใหญ่ 60*46*30 cm / 20 kg,1,กล่องโฟม,20.0,60.0,46.0,30.0,กล่องเย็น,overweight,1


In [ ]:
df_summary = df_repeat['PACKAGE']+' '+df_repeat['type']

In [ ]:
df_summary.value_counts()

,count
กล่อง overweight,326465
กล่องเย็น overweight,25759
เหมา overweight,4094
เอกสาร 1,906
กล่องกันกระแทก overweight,19
กล่อง 7,11
กล่องเย็น 6,10
กล่อง 6,9
กล่อง 8,5


In [ ]:
data = df_merged.copy()
data['PACKAGE'] = df_summary
data = data.drop('type',axis=1)
data.dropna(inplace=True)
data

,BILL_INDEX,BILL_DATE,ITEM_DESCแสดงหน้าบิล,QTY,UNIT_NAME,weight (kg),กว้าง (cm),ยาว (cm),สูง (cm),PACKAGE
268,414,20240301,ของสด กล่องโฟมใหญ่ 60*46*30 cm / 20 kg ...,1,โฟม ญ ...,20.0,60.0,46.0,30.0,กล่องเย็น overweight
269,415,20240301,ของสด กล่องโฟมใหญ่ 60*46*30 cm / 20 kg ...,1,ฟญ ...,20.0,60.0,46.0,30.0,กล่องเย็น overweight
314,517,20240301,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg ...,1,โฟม ก ...,15.0,35.0,48.0,35.0,กล่องเย็น overweight
318,523,20240301,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg ...,1,โฟม ก ...,15.0,35.0,48.0,35.0,กล่องเย็น overweight
325,530,20240301,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg ...,1,โฟม ก ...,15.0,35.0,48.0,35.0,กล่องเย็น overweight
...,...,...,...,...,...,...,...,...,...,...
76749,110322,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg ...,1.0,โฟมกลาง ...,15.0,35.0,48.0,35.0,กล่องเย็น overweight
76750,110326,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg-ลูกชิ้น...,1.0,โฟมกลาง ...,15.0,35.0,48.0,35.0,กล่องเย็น overweight
76751,110334,20240201,ของสด กล่องโฟมจิ๋ว 37*25*21 cm / 2 kg ...,2.0,โฟมเล็ก ...,2.0,37.0,25.0,21.0,กล่องเย็น 6
76752,110436,20240201,ของสด กล่องโฟมกลาง 35*48*35 cm / 15 kg-ทูน่า ...,1.0,โฟมกลาง ...,15.0,35.0,48.0,35.0,กล่องเย็น overweight


In [ ]:
data.to_csv('data_final.csv', index=False)